## Load Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

## Define Transformations

In [3]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

## Load the Data

In [4]:
train_data=datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_data=datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,  
    transform=transform
)

In [5]:
train_loader=DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True
)

In [6]:
test_loader=DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=False
)

## Build the Architecture

In [7]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar,self).__init__()
        self.cnn1=nn.Conv2d(3,6,5)
        self.cnn2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(5*5*16,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x):
        x=self.cnn1(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x,2)

        x=self.cnn2(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x,2)
        
        x=torch.flatten(x,1)
        x=self.fc1(x)
        x=torch.relu(x)
        x=self.fc2(x)
        x=torch.relu(x)
        x=self.fc3(x)
        return x
            

## Object

In [8]:
model=cnn_cifar()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)





## Training

In [11]:
for epoch in range(10):
    for images,labels in train_loader:
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')
    

Epoch [1/10], Loss: 0.6047
Epoch [2/10], Loss: 0.7645
Epoch [3/10], Loss: 1.1458
Epoch [4/10], Loss: 1.5153
Epoch [5/10], Loss: 0.7219
Epoch [6/10], Loss: 0.6921
Epoch [7/10], Loss: 0.9031
Epoch [8/10], Loss: 0.6139
Epoch [9/10], Loss: 0.7920
Epoch [10/10], Loss: 0.4483


In [16]:
model.eval()
correct=0
total=0
with torch.no_grad():
    for images,labels in test_loader:
        outputs=model(images)
        _,predictions=torch.max(outputs,1)
        correct+=(predictions==labels).sum().item()
        total+=labels.size(0)
print(f'Accuracy: {(correct/total)*100}')



        



Accuracy: 62.949999999999996
